Copyright 2021-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Anisotropic tutorial with optimization loop chassis

This notebook walks through the [Anisotropic Metric Tutorial](../docs/examples/anisotropic_tutorial.ipynb) using experimental optimaztion loop chassis. The goal is to recover the response on the held-out test data by training a simple anisotropic `MuyGPS` model on the perturbed training data with `nu` smoothness hyperparameter known, while the two `distance scaling` hyperparameters are to be learned.
1. Sample a 2D surface from a conventional GP
2. Create duplicate of `MuyGPyS` object used in `sampler` to be used for training and inference
3. Experiments using `optimize_from_tensors_mini_batch` to create nearest neighbors lookup, sample batches of data, construct tensors, generate objective function, and run bayes optimization using numpy math backend
4. Inference (best performer)
5. Analysis (best performer)

In [ ]:
#TODO import sys
# for m in sys.modules.keys():
#     if m.startswith("Muy"):
#         sys.modules.pop(m)
# %env MUYGPYS_BACKEND=numpy
# %env MUYGPYS_FTYPE=64

import numpy as np

from docs.examples.utils import UnivariateSampler2D

from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.distortion import AnisotropicDistortion, l2
from MuyGPyS.gp.hyperparameter import ScalarHyperparameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.optimize.experimental.chassis import optimize_from_tensors_mini_batch


np.random.seed(0)

points_per_dim = 60                             # Observations per dimension
train_step = 13                                 # Train/test data split
nugget_noise = HomoscedasticNoise(1e-14)        # Assume no noise in truth
measurement_noise = HomoscedasticNoise(1e-7)    # Noise to perturb train
sim_nu = ScalarHyperparameter(1.5)              # HP smoothness
sim_length_scale0 = ScalarHyperparameter(0.1)   # HP distance scaling dim 0
sim_length_scale1 = ScalarHyperparameter(0.5)   # HP distance scaling dim 1
sampler = UnivariateSampler2D(
    points_per_dim=points_per_dim,
    train_step=train_step,
    kernel=Matern(
        nu=sim_nu,
        metric=AnisotropicDistortion(
            l2,
            length_scale0=sim_length_scale0,
            length_scale1=sim_length_scale1,
        ),
    ),
    eps=nugget_noise,
    measurement_eps=measurement_noise,
)
train_features, test_features = sampler.features()
train_responses, test_responses = sampler.sample()
# sampler.plot_sample()#TODO

exp_length_scale0 = ScalarHyperparameter("log_sample", (0.01, 1.0))
exp_length_scale1 = ScalarHyperparameter("log_sample", (0.01, 1.0))
muygps = MuyGPS(
    kernel=Matern(
        nu=sim_nu,
        metric=AnisotropicDistortion(
            l2,
            length_scale0=exp_length_scale0,
            length_scale1=exp_length_scale1,
        ),
    ),
    eps=measurement_noise,
)


Experiment #1 (baseline) - validate following [Anisotropic Metric Tutorial](../docs/examples/anisotropic_tutorial.ipynb) using all training data and equivalent number of planned Bayesian optimization iterations for experiments 2 & 3 that utilize optimiztion loop.

In [ ]:
muygps_tutorial = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=sampler.train_count,
    train_count=sampler.train_count,
    num_epochs=1,
    batch_features=None,
    loss_method="lool",
    obj_method="loo_crossval",
    sigma_method="analytic",
    verbose=False,#True,
    random_state=1,
    init_points=5,
    n_iter=20,
    allow_duplicate_points=True,
)

print(f"BayesianOptimization finds an optimimal pair of length scales")
print(f"\tlength_scale0 is {muygps_tutorial.kernel.distortion_fn.length_scale['length_scale0']()}")
print(f"\tlength_scale1 is {muygps_tutorial.kernel.distortion_fn.length_scale['length_scale1']()}")
print(f"Optimized anisotropic sigma_sq: {muygps_tutorial.sigma_sq()}")


Experiment #2 - optimization loop

In [ ]:
batch_count=55
num_epochs=int(sampler.train_count / batch_count)
print(f"total sample size {sampler.train_count}")
print(f"sample batch size {batch_count}")
print(f"num epochs {num_epochs}")
muygps_optloop = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=batch_count,
    train_count=sampler.train_count,
    num_epochs=num_epochs,
    batch_features=None,
    loss_method="lool",
    obj_method="loo_crossval",
    sigma_method="analytic",
    verbose=False,# True,
    random_state=1,
    init_points=5,
    n_iter=20,
    allow_duplicate_points=True,
)
print(f"BayesianOptimization finds an optimimal pair of length scales")
print(f"\tlength_scale0 is {muygps_optloop.kernel.distortion_fn.length_scale['length_scale0']()}")
print(f"\tlength_scale1 is {muygps_optloop.kernel.distortion_fn.length_scale['length_scale1']()}")
print(f"Optimized anisotropic sigma_sq: {muygps_optloop.sigma_sq()}")


Experiment #3 - optimization loop + learned distance scaling HPs

In [ ]:
metric = AnisotropicDistortion(
    l2,
    length_scale0=sim_length_scale0,
    length_scale1=sim_length_scale1,
)
result = metric(
    train_features,
    length_scale0=sim_length_scale0,
    length_scale1=sim_length_scale1,
)